In [ ]:
!pip install meteostat

In [ ]:
# Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from meteostat import Point, Daily

# Set plot style
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load Data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

### 2. Preprocessing

#### Date and Time Features
- [ ] **Extract Date Features**: Extract year, month, day, and hour from the timestamp for each entry.
- [ ] **Cyclic Features**: Add cyclic features for hour and month using sine and cosine transformations.

#### Lag and Rolling Features
- [ ] **Lagged Values**: Create lagged features for each pollutant (1, 24, 168 hours to capture short, daily, and weekly patterns).
- [ ] **Rolling Statistics**: Add rolling mean, rolling standard deviation, and rolling min/max features to smooth trends.

#### Handle Missing Values
- [ ] **Impute Missing Values**: Choose and implement a strategy (e.g., forward-fill, backward-fill, interpolation) for missing data in each pollutant.
- [ ] **Drop or Flag Missing Rows**: If a significant portion of the data is missing, consider dropping those rows or creating an indicator feature to mark them.

#### External Weather Data (Optional but Recommended)
- [ ] **Incorporate Weather Data**: If available, obtain weather data for Paris (temperature, humidity, wind speed).
- [ ] **Merge with Pollutant Data**: Align weather data with your time series dataset based on the timestamp.

#### Data Scaling
- [ ] **Scale Features**: Use standard scaling or min-max scaling for numerical features, especially for pollutants and weather variables (if any).

#### Target Preparation
- [ ] **Prepare Target Variables**: Shift the pollutant columns to create target variables aligned with the forecasting horizon (e.g., next hour, next day).
- [ ] **Split Data**: Divide the data into training and validation sets, ensuring the split respects the time sequence.




In [ ]:
# Handling Missing Values

# Replace inf/-inf with NaN (if any)
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Impute missing values (using median as it's robust to outliers)
imputer = SimpleImputer(strategy='median')
train_df = pd.DataFrame(imputer.fit_transform(train_df), columns=train_df.columns, index=train_df.index)

In [ ]:
# Extracting Date Information
for df in [train_df, test_df]:
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['day_of_week'] = df.index.dayofweek
    df['hour'] = df.index.hour

# Create Lag Features (e.g., lags of 1, 3, 6, and 24 hours)
lags = [1, 3, 6, 24]
for lag in lags:
    for col in ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']:
        train_df[f'{col}_lag_{lag}'] = train_df[col].shift(lag)

# Drop rows with NaN values introduced by lags (at the beginning of the data)
train_df.dropna(inplace=True)

In [ ]:
# Scaling/Normalization

# Select features for scaling (all pollutant columns and lags)
scaler = StandardScaler()
pollutant_columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25'] 
pollutant_columns = pollutant_columns + [f'{col}_lag_{lag}' for col in pollutant_columns for lag in lags]
train_df[pollutant_columns] = scaler.fit_transform(train_df[pollutant_columns])

In [ ]:
# Adding External Weather Data

# Define the time period for which we need the data
start_date = train_df.index.min()
end_date = train_df.index.max()

# Define the location (Paris coordinates)
paris = Point(48.8566, 2.3522)

# Fetch daily historical data for Paris from Meteostat
weather_data = Daily(paris, start_date, end_date)
weather_data = weather_data.fetch()

# Display the first few rows of the weather data
print(weather_data.head())

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2020-01-02   7.3   4.3   9.2   1.3   NaN  186.0   8.1  35.0  1026.4   NaN
2020-01-03   9.8   8.5  12.2   0.5   NaN  249.0  15.7  41.0  1024.8   NaN
2020-01-04   7.3   4.3  11.0   0.3   NaN  309.0   9.7  33.0  1035.8   NaN
2020-01-05   7.8   7.1   9.1   0.0   NaN  265.0   5.4  21.0  1036.3   NaN
2020-01-06   4.7   1.6   8.3   0.0   NaN  180.0   9.5  55.0  1027.6   NaN


In [ ]:
# Reset index in `weather_data` to use the date as a regular column
weather_data = weather_data.reset_index()
weather_data['weather_time'] = weather_data['time'].dt.floor('D')  # Convert to daily frequency

train_df = train_df.reset_index()
train_df['air_quality_time'] = train_df['id'].dt.floor('D')

# Merge the weather data with your air quality data
merged_df = pd.merge(train_df, weather_data, left_on='air_quality_time', right_on='weather_time', how='left')

# Drop unnecessary columns and clean up
merged_df = merged_df.drop(columns=['air_quality_time', 'weather_time'])

# Save the combined DataFrame to a new CSV file
merged_df.to_csv('../data/train_with_weather.csv', index=False)

In [ ]:
# Target Variable Extraction

train_processed = pd.read_csv('../data/train_with_weather.csv')

target_columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

# Separate features and target for training
X_train = train_processed.drop(columns=target_columns)
y_train = train_processed[target_columns]

# Test set only has features, as targets are unknown
X_test = test_df.reset_index()

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (40967, 37)
y_train shape: (40967, 5)
X_test shape: (504, 6)


Need to do the same preprocessing for X_test to get the same columns as X_train